# Step 2: Location_Query
###### Let's find some locations... a lot of them. 
###### But for the love of all that is holy, do not run this again because running several thousand Google API calls is EXPENSIVE. 
###### Thank goodness for the free trial credit is all I'm saying. Whew.
-----
Now we're going to cross reference some of the data we exported to CSV from step 1.

Using the centerpoint of the zipcodes we're visualizing, we will perform a nested loop of queries against those coordinates increasing our query range on each pass. (1km, 2km, and 3km). Once we have the returns, we use the Google API ID as a unique identifier to eliminate duplicates from the compiled list.

Through testing, it was found that assembling a master list by running multiple queries, using the google API next-page capability and removing duplicates after completion resulted in the most thourough query results.

As the query structure involves making several hundred api calls: (3 resolutions * 81 zipcodes * 2 api calls = 486), it does take time to run, and ultimately is being saved into a CSV format for later reference wihtout the need for unnecessary additional calls to the Google Places API. The query, run on April 22nd, 2019, returned 3820 unique results with type "Doctor".

Annoyingly, Google did not return those 3820 zipcodes along with the rest in the 'nearby' search results. This required a followup loop which pulls down location details on each item in the list by the place_id. This is then used to attach a list to the dataframe, allowing the doctors to be sorted by zipcode for later analysis.

In [1]:
import requests
import pandas as pd
import gmaps
import json
import csv
import time
import os
from keys import gapikey
from area import area
import ast
import re

with open('resources/zip_data.csv', newline='\n') as b:
    ziptgt = pd.read_csv(b)
with open('resources/zip_coords.csv', newline='\n', encoding='utf-8-sig') as c:
    zip_coords = pd.read_csv(c)

In [2]:
ziptgt = ziptgt.set_index('zip')
zip_coords = zip_coords.set_index('ZIP')
zip_df = ziptgt.join(zip_coords, how='left')

zip_df.head()

,city,county,median_household_income,population,feature,area,density,nml_dens,nml_inc,color_key,LAT,LNG
zip,,,,,,,,,,,,
20001,"Washington, DC",District Of Columbia,100447.0,41692,"{'type': 'Feature', 'geometry': {'type': 'Poly...",5.835524,7144.516970,0.759751,0.386274,#0062ce,38.910353,-77.017739
20002,"Washington, DC",District Of Columbia,82022.0,52867,"{'type': 'Feature', 'geometry': {'type': 'Poly...",14.228399,3715.597280,0.964587,0.284281,#0048db,38.905026,-76.983607
20003,"Washington, DC",District Of Columbia,118607.0,26330,"{'type': 'Feature', 'geometry': {'type': 'Poly...",6.447784,4083.573672,0.478169,0.486800,#007cc1,38.881904,-76.990935
20004,"Washington, DC",District Of Columbia,144583.0,1610,"{'type': 'Feature', 'geometry': {'type': 'Mult...",0.904905,1779.192084,0.025057,0.630593,#00a1ae,38.894885,-77.028638
20005,"Washington, DC",District Of Columbia,94506.0,12311,"{'type': 'Feature', 'geometry': {'type': 'Poly...",1.122787,10964.678013,0.221204,0.353387,#005ad2,38.904674,-77.031578


In [3]:
# !!! DO NOT RUN THIS AGAIN (THIS IS A VERY EXPENSIVE QUERY USE THE CSVs) !!!
# places_df = pd.DataFrame(columns=['geometry','icon','id', 'name', 'opening_hours', 'photos', 'place_id', 'plus_code', 'rating', 'reference', 'scope', 'types', 'user_ratings_total', 'vicinity'])

# i = 0
# print("Queries have started. Might be a good time to go get some coffee, it's going to take a few minutes")
# radius_list = [1000, 2000, 3000]
# for radius in radius_list:
#     print(f'Starting {radius} meter scan.')
#     for pzip in list(zip_df.index.values):
#         try:
#             npt = 0
#             target_geo = f"{zip_df['LAT'][pzip]},{zip_df['LNG'][pzip]}"
#             target_type = 'doctor'
#             target_radius = radius

#             params1 = {
#                 "location": target_geo,
#                 "radius": target_radius,
#                 "type": target_type,
#                 "key": gapikey,
#             }

#             base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#             response = requests.get(base_url, params=params1)

#             data1 = response.json()

#             npt = data1['next_page_token']

#             params2 = {
#                 "key": gapikey,
#                 "pagetoken": npt
#             }

#             time.sleep(2)

#             response = requests.get(f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?key={gapikey}&pagetoken={npt}')

#             data2 = response.json()

#             places_df = places_df.append(data1['results'], ignore_index=True)
#             places_df = places_df.append(data2['results'], ignore_index=True)
#             i += 1
#             os.system('clear')
#             print(f'places retrieved for {pzip} | {i} of {len(zip_df.index)} @ {radius} meters')
#             print(f'{len(places_df.index)} places retrieved')
#         except KeyError:
#           print(f'key error, next zip')

#     places_df = places_df.drop_duplicates(subset='id').reset_index(drop=True)
    
# print('Query complete. Thanks for waiting.')

Queries have started. Might be a good time to go get some coffee, it's going to take a few minutes
Starting 1000 meter scan.


/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


places retrieved for 20001 | 1 of 81 @ 1000 meters
40 places retrieved
places retrieved for 20002 | 2 of 81 @ 1000 meters
80 places retrieved
places retrieved for 20003 | 3 of 81 @ 1000 meters
120 places retrieved
places retrieved for 20004 | 4 of 81 @ 1000 meters
160 places retrieved
places retrieved for 20005 | 5 of 81 @ 1000 meters
200 places retrieved
places retrieved for 20006 | 6 of 81 @ 1000 meters
240 places retrieved
places retrieved for 20007 | 7 of 81 @ 1000 meters
280 places retrieved
places retrieved for 20008 | 8 of 81 @ 1000 meters
320 places retrieved
places retrieved for 20009 | 9 of 81 @ 1000 meters
360 places retrieved
places retrieved for 20010 | 10 of 81 @ 1000 meters
400 places retrieved
places retrieved for 20011 | 11 of 81 @ 1000 meters
436 places retrieved
places retrieved for 20012 | 12 of 81 @ 1000 meters
476 places retrieved
key error, next zip
places retrieved for 20016 | 13 of 81 @ 1000 meters
516 places retrieved
places retrieved for 20017 | 14 of 81 @ 10

places retrieved for 22042 | 105 of 81 @ 2000 meters
3894 places retrieved
places retrieved for 22043 | 106 of 81 @ 2000 meters
3934 places retrieved
places retrieved for 22044 | 107 of 81 @ 2000 meters
3974 places retrieved
places retrieved for 22046 | 108 of 81 @ 2000 meters
4014 places retrieved
places retrieved for 22101 | 109 of 81 @ 2000 meters
4054 places retrieved
places retrieved for 22201 | 110 of 81 @ 2000 meters
4094 places retrieved
places retrieved for 22202 | 111 of 81 @ 2000 meters
4134 places retrieved
places retrieved for 22203 | 112 of 81 @ 2000 meters
4174 places retrieved
places retrieved for 22204 | 113 of 81 @ 2000 meters
4214 places retrieved
places retrieved for 22205 | 114 of 81 @ 2000 meters
4254 places retrieved
places retrieved for 22206 | 115 of 81 @ 2000 meters
4294 places retrieved
places retrieved for 22207 | 116 of 81 @ 2000 meters
4334 places retrieved
places retrieved for 22209 | 117 of 81 @ 2000 meters
4374 places retrieved
places retrieved for 2221

In [4]:
places_geo = pd.DataFrame(columns=['lat','lng','name','vicinity','placeid'])

for p in list(places_df.index.values):
    lat = places_df['geometry'][p]['location']['lat']
    lng = places_df['geometry'][p]['location']['lng']
    vicinity = places_df['vicinity'][p]
    name = places_df['name'][p]
    place_id = places_df['place_id'][p]
    geo_tmp = pd.DataFrame([[lat,lng,name,vicinity,place_id]],columns=['lat','lng','name','vicinity','placeid'])
    places_geo = places_geo.append(geo_tmp).reset_index(drop=True)

places_geo.head()

,lat,lng,name,vicinity,placeid
0,38.917568,-77.020726,Howard University Hospital:pulmonary,"2041 Georgia Avenue Northwest, Washington",ChIJf2TW1OS3t4kRmYwM_D3YU9g
1,38.905312,-77.012765,"Shiban Raina, MD","128 M Street Northwest #050, Washington",ChIJqVHvjcHHt4kR9bkSKfeYk6M
2,38.905363,-77.018377,Aziz Salim MD,"2175 K Street Northwest, Washington",ChIJO7BERbS3t4kRdCq1JjRVF9c
3,38.910311,-77.021587,Abramson Randi C MD,"1525 7th Street Northwest, Washington",ChIJNcEa_fG3t4kROr29e7EFZJw
4,38.910311,-77.021587,"Hope Ferdowsian, MD, MPH","1525 7th Street Northwest, Washington",ChIJxfO-cZTJt4kR1eicxoihlSc


In [5]:
places_geo.to_csv('resources/places_geo_multi.csv', encoding='utf-8')

In [6]:
with open('resources/places_geo_multi.csv', newline='\n') as b:
    geo_places = pd.read_csv(b)
    
places_geo.head()

,lat,lng,name,vicinity,placeid
0,38.917568,-77.020726,Howard University Hospital:pulmonary,"2041 Georgia Avenue Northwest, Washington",ChIJf2TW1OS3t4kRmYwM_D3YU9g
1,38.905312,-77.012765,"Shiban Raina, MD","128 M Street Northwest #050, Washington",ChIJqVHvjcHHt4kR9bkSKfeYk6M
2,38.905363,-77.018377,Aziz Salim MD,"2175 K Street Northwest, Washington",ChIJO7BERbS3t4kRdCq1JjRVF9c
3,38.910311,-77.021587,Abramson Randi C MD,"1525 7th Street Northwest, Washington",ChIJNcEa_fG3t4kROr29e7EFZJw
4,38.910311,-77.021587,"Hope Ferdowsian, MD, MPH","1525 7th Street Northwest, Washington",ChIJxfO-cZTJt4kR1eicxoihlSc


In [13]:
# !!! DO NOT RUN THIS AGAIN EITHER (THIS IS AN EVEN MORE EXPENSIVE QUERY USE THE CSVs) !!!
# place_zip = []
# expt_zip = []
# i = 1
# for placeid in geo_places['placeid']:
#     params3 = {
#         "placeid":placeid,
#         "key": gapikey,
#     }
#     mtchd = False
#     base_url = "https://maps.googleapis.com/maps/api/place/details/json"
#     response = requests.get(base_url, params=params3)
#     place_details = response.json()
#     print(f"processing {i} of {len(geo_places['placeid'])}")
#     i += 1
#     for addcomp in place_details['result']['address_components']:
#         match = re.match(r'^2\d{4}$', addcomp['short_name'])
#         if match:
#             if mtchd == False:
#                 place_zip.append(match.string)
#                 mtchd = True
#                 mtch1 = match.string
#             else:
#                 expt_zip.append(match.string)
#                 print(f"possible double match {match.string} & {mtch1}")

processing 1 of 3819
processing 2 of 3819
processing 3 of 3819
processing 4 of 3819
processing 5 of 3819
processing 6 of 3819
processing 7 of 3819
processing 8 of 3819
processing 9 of 3819
processing 10 of 3819
processing 11 of 3819
processing 12 of 3819
processing 13 of 3819
processing 14 of 3819
processing 15 of 3819
processing 16 of 3819
processing 17 of 3819
processing 18 of 3819
processing 19 of 3819
processing 20 of 3819
processing 21 of 3819
processing 22 of 3819
processing 23 of 3819
processing 24 of 3819
processing 25 of 3819
processing 26 of 3819
processing 27 of 3819
processing 28 of 3819
processing 29 of 3819
processing 30 of 3819
processing 31 of 3819
processing 32 of 3819
processing 33 of 3819
processing 34 of 3819
processing 35 of 3819
processing 36 of 3819
processing 37 of 3819
processing 38 of 3819
processing 39 of 3819
processing 40 of 3819
processing 41 of 3819
processing 42 of 3819
processing 43 of 3819
processing 44 of 3819
processing 45 of 3819
processing 46 of 38

processing 362 of 3819
processing 363 of 3819
processing 364 of 3819
processing 365 of 3819
processing 366 of 3819
processing 367 of 3819
processing 368 of 3819
processing 369 of 3819
processing 370 of 3819
processing 371 of 3819
processing 372 of 3819
processing 373 of 3819
processing 374 of 3819
processing 375 of 3819
processing 376 of 3819
processing 377 of 3819
processing 378 of 3819
processing 379 of 3819
processing 380 of 3819
processing 381 of 3819
processing 382 of 3819
processing 383 of 3819
processing 384 of 3819
processing 385 of 3819
processing 386 of 3819
processing 387 of 3819
processing 388 of 3819
processing 389 of 3819
processing 390 of 3819
processing 391 of 3819
processing 392 of 3819
processing 393 of 3819
processing 394 of 3819
processing 395 of 3819
processing 396 of 3819
processing 397 of 3819
processing 398 of 3819
processing 399 of 3819
processing 400 of 3819
processing 401 of 3819
processing 402 of 3819
processing 403 of 3819
processing 404 of 3819
processing 

processing 719 of 3819
processing 720 of 3819
processing 721 of 3819
processing 722 of 3819
processing 723 of 3819
processing 724 of 3819
processing 725 of 3819
processing 726 of 3819
processing 727 of 3819
processing 728 of 3819
processing 729 of 3819
processing 730 of 3819
processing 731 of 3819
processing 732 of 3819
processing 733 of 3819
processing 734 of 3819
processing 735 of 3819
processing 736 of 3819
processing 737 of 3819
processing 738 of 3819
processing 739 of 3819
processing 740 of 3819
processing 741 of 3819
processing 742 of 3819
processing 743 of 3819
processing 744 of 3819
processing 745 of 3819
processing 746 of 3819
processing 747 of 3819
processing 748 of 3819
processing 749 of 3819
processing 750 of 3819
processing 751 of 3819
processing 752 of 3819
processing 753 of 3819
processing 754 of 3819
processing 755 of 3819
processing 756 of 3819
processing 757 of 3819
processing 758 of 3819
processing 759 of 3819
processing 760 of 3819
processing 761 of 3819
processing 

processing 1074 of 3819
processing 1075 of 3819
processing 1076 of 3819
processing 1077 of 3819
processing 1078 of 3819
processing 1079 of 3819
processing 1080 of 3819
processing 1081 of 3819
processing 1082 of 3819
processing 1083 of 3819
processing 1084 of 3819
processing 1085 of 3819
processing 1086 of 3819
processing 1087 of 3819
processing 1088 of 3819
processing 1089 of 3819
processing 1090 of 3819
processing 1091 of 3819
processing 1092 of 3819
processing 1093 of 3819
processing 1094 of 3819
processing 1095 of 3819
processing 1096 of 3819
processing 1097 of 3819
processing 1098 of 3819
processing 1099 of 3819
processing 1100 of 3819
processing 1101 of 3819
processing 1102 of 3819
processing 1103 of 3819
processing 1104 of 3819
processing 1105 of 3819
processing 1106 of 3819
processing 1107 of 3819
processing 1108 of 3819
processing 1109 of 3819
processing 1110 of 3819
processing 1111 of 3819
processing 1112 of 3819
processing 1113 of 3819
processing 1114 of 3819
processing 1115 

processing 1416 of 3819
processing 1417 of 3819
processing 1418 of 3819
processing 1419 of 3819
processing 1420 of 3819
processing 1421 of 3819
processing 1422 of 3819
processing 1423 of 3819
processing 1424 of 3819
processing 1425 of 3819
processing 1426 of 3819
processing 1427 of 3819
processing 1428 of 3819
processing 1429 of 3819
processing 1430 of 3819
processing 1431 of 3819
processing 1432 of 3819
processing 1433 of 3819
processing 1434 of 3819
processing 1435 of 3819
processing 1436 of 3819
processing 1437 of 3819
processing 1438 of 3819
processing 1439 of 3819
processing 1440 of 3819
processing 1441 of 3819
processing 1442 of 3819
processing 1443 of 3819
processing 1444 of 3819
processing 1445 of 3819
processing 1446 of 3819
processing 1447 of 3819
processing 1448 of 3819
processing 1449 of 3819
processing 1450 of 3819
processing 1451 of 3819
processing 1452 of 3819
processing 1453 of 3819
processing 1454 of 3819
processing 1455 of 3819
processing 1456 of 3819
processing 1457 

...

processing 2442 of 3819
processing 2443 of 3819
processing 2444 of 3819
processing 2445 of 3819
processing 2446 of 3819
processing 2447 of 3819
processing 2448 of 3819
processing 2449 of 3819
processing 2450 of 3819
processing 2451 of 3819
processing 2452 of 3819
processing 2453 of 3819
processing 2454 of 3819
processing 2455 of 3819
processing 2456 of 3819
processing 2457 of 3819
processing 2458 of 3819
processing 2459 of 3819
processing 2460 of 3819
processing 2461 of 3819
processing 2462 of 3819
processing 2463 of 3819
processing 2464 of 3819
processing 2465 of 3819
processing 2466 of 3819
processing 2467 of 3819
processing 2468 of 3819
processing 2469 of 3819
processing 2470 of 3819
processing 2471 of 3819
processing 2472 of 3819
processing 2473 of 3819
processing 2474 of 3819
processing 2475 of 3819
processing 2476 of 3819
processing 2477 of 3819
processing 2478 of 3819
processing 2479 of 3819
processing 2480 of 3819
processing 2481 of 3819
processing 2482 of 3819
processing 2483 

processing 2784 of 3819
processing 2785 of 3819
processing 2786 of 3819
processing 2787 of 3819
processing 2788 of 3819
processing 2789 of 3819
processing 2790 of 3819
processing 2791 of 3819
processing 2792 of 3819
processing 2793 of 3819
processing 2794 of 3819
processing 2795 of 3819
processing 2796 of 3819
processing 2797 of 3819
processing 2798 of 3819
processing 2799 of 3819
processing 2800 of 3819
processing 2801 of 3819
processing 2802 of 3819
processing 2803 of 3819
processing 2804 of 3819
processing 2805 of 3819
processing 2806 of 3819
processing 2807 of 3819
processing 2808 of 3819
processing 2809 of 3819
processing 2810 of 3819
processing 2811 of 3819
processing 2812 of 3819
processing 2813 of 3819
processing 2814 of 3819
processing 2815 of 3819
processing 2816 of 3819
processing 2817 of 3819
processing 2818 of 3819
processing 2819 of 3819
processing 2820 of 3819
processing 2821 of 3819
processing 2822 of 3819
processing 2823 of 3819
processing 2824 of 3819
processing 2825 

processing 3127 of 3819
processing 3128 of 3819
processing 3129 of 3819
processing 3130 of 3819
processing 3131 of 3819
processing 3132 of 3819
processing 3133 of 3819
processing 3134 of 3819
processing 3135 of 3819
processing 3136 of 3819
processing 3137 of 3819
processing 3138 of 3819
processing 3139 of 3819
processing 3140 of 3819
processing 3141 of 3819
processing 3142 of 3819
processing 3143 of 3819
processing 3144 of 3819
processing 3145 of 3819
processing 3146 of 3819
processing 3147 of 3819
processing 3148 of 3819
processing 3149 of 3819
processing 3150 of 3819
processing 3151 of 3819
processing 3152 of 3819
processing 3153 of 3819
processing 3154 of 3819
processing 3155 of 3819
processing 3156 of 3819
processing 3157 of 3819
processing 3158 of 3819
processing 3159 of 3819
processing 3160 of 3819
processing 3161 of 3819
processing 3162 of 3819
processing 3163 of 3819
processing 3164 of 3819
processing 3165 of 3819
processing 3166 of 3819
processing 3167 of 3819
processing 3168 

processing 3469 of 3819
processing 3470 of 3819
processing 3471 of 3819
processing 3472 of 3819
processing 3473 of 3819
processing 3474 of 3819
processing 3475 of 3819
processing 3476 of 3819
processing 3477 of 3819
processing 3478 of 3819
processing 3479 of 3819
processing 3480 of 3819
processing 3481 of 3819
processing 3482 of 3819
processing 3483 of 3819
processing 3484 of 3819
processing 3485 of 3819
processing 3486 of 3819
processing 3487 of 3819
processing 3488 of 3819
processing 3489 of 3819
processing 3490 of 3819
processing 3491 of 3819
processing 3492 of 3819
processing 3493 of 3819
processing 3494 of 3819
processing 3495 of 3819
processing 3496 of 3819
processing 3497 of 3819
processing 3498 of 3819
processing 3499 of 3819
processing 3500 of 3819
processing 3501 of 3819
processing 3502 of 3819
processing 3503 of 3819
processing 3504 of 3819
processing 3505 of 3819
processing 3506 of 3819
processing 3507 of 3819
processing 3508 of 3819
processing 3509 of 3819
processing 3510 

processing 3812 of 3819
processing 3813 of 3819
processing 3814 of 3819
processing 3815 of 3819
processing 3816 of 3819
processing 3817 of 3819
processing 3818 of 3819
processing 3819 of 3819


In [27]:
places_geo2.to_csv('resources/places_geo2.csv', encoding='utf-8')
places_geo2.head()

,lat,lng,name,vicinity,placeid,zip
0,38.917568,-77.020726,Howard University Hospital:pulmonary,"2041 Georgia Avenue Northwest, Washington",ChIJf2TW1OS3t4kRmYwM_D3YU9g,20001
1,38.905312,-77.012765,"Shiban Raina, MD","128 M Street Northwest #050, Washington",ChIJqVHvjcHHt4kR9bkSKfeYk6M,20001
2,38.905363,-77.018377,Aziz Salim MD,"2175 K Street Northwest, Washington",ChIJO7BERbS3t4kRdCq1JjRVF9c,20001
3,38.910311,-77.021587,Abramson Randi C MD,"1525 7th Street Northwest, Washington",ChIJNcEa_fG3t4kROr29e7EFZJw,20001
4,38.910311,-77.021587,"Hope Ferdowsian, MD, MPH","1525 7th Street Northwest, Washington",ChIJxfO-cZTJt4kR1eicxoihlSc,20001


In [25]:
gmaps.configure(api_key=gapikey)

places = places_geo[['lat', 'lng']]

places_layer = gmaps.symbol_layer(
    places, fill_color='red', stroke_color='red', scale=1)
fig = gmaps.figure()
fig.add_layer(places_layer)
fig




Figure(layout=FigureLayout(height='420px'))